In [27]:
# Dependencies 
import os
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import pandas as pd
from flask import Flask, redirect, render_template, jsonify
import time
from selenium import webdriver
import lxml
import urllib

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
db = client.quotes
collection = db.quotes

#browser=Browser("chrome", headless=False)

total_quotes=[]
author_list=[]
__id = 0 #counter

for r in range(1,11):
    url=f'http://quotes.toscrape.com/page/{r}'
    response=requests.get(url)
    #browser.visit(url)
    soup=bs(response.text, "lxml")

    ascrape=soup.find_all('div',{'class':'quote'})
    for a in ascrape:
        __id += 1 
        tags_list=[]
        quote=a.find('span', class_='text').text
        author_name=a.find('small', class_='author').text
        tags=a.find('div', class_='tags').find_all('a')
        for tag in tags:
            tag_text=tag.text.strip()
            tags_list.append(tag_text)
        #one_quote=[text,author,tags_list]
        #total_quotes.append(one_quote)
        href = a.a["href"]
        author_url = f'http://quotes.toscrape.com{href}'
        author_response = requests.get(author_url)
        author_soup = bs(author_response.text, 'lxml')
        author_born = author_soup.find('span', class_ = 'author-born-date').text
        author_desrp = author_soup.find('div', class_ = 'author-description').text
        author = {
            "name": author_name,
            "birthdate": author_born,
            "description": author_desrp
        }
        data = {
            '__id': __id,
            'quote': quote,
            'author': author,
            'tags': tags_list,
        }
        total_quotes.append(data)
        collection.insert_one(data)


        

#     about_links=browser.find_by_text("(about)")
#     href_list=[]
#     for l in about_links:
#         ref=l['href']
#         href_list.append(ref)
#     for h in href_list:
#         print(h)
#         #browser.visit(h)
#         req=requests.get(h)
#         soup=bs(req.text, "lxml")
#         author_birth=soup.find('span', class_="author-born-date").text
#         author_desc=soup.find('div', class_="author-description").text
#         author_name=soup.find('h3', class_="author-title").text
#         author_dict={'Name':author_name.strip(), 'Birthdate': author_birth.strip(), 'Bio':author_desc.strip()}
#         author_list.append(author_dict)
        
#     quote_info = {
#         'quote_text': text,
#         'author': author,
#         'tags': tags_list, 
#         'author_birthdate': author_birth,
#         'author_description': author_desc
#     }
#     print(quote_info)

In [28]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [29]:
# Define database and collection 
db = client.quote_db
collection = db.quotes

In [30]:
# URL of page to be scraped 
url = 'http://quotes.toscrape.com'

# Retrieve page with the requests module 
response = requests.get(url)

# Create BeautifulSoup object parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [31]:
# pages = np.arange(1,10,1)
# for page in pages:

# Retrieve the parent divs for all quotes 
results = soup.find_all('div', class_ = 'quote')
scrapped = []


# Loop over results to get quote data 
for result in results: 
    # Scrape Quote Text 
    quote_text = result.find('span', class_ = 'text').text
    
    # Scrage Tags 
    tags = result.find('div', class_ = 'tags').text
    
    # Scrape Author Name 
    author_name = result.find('small', class_ = 'author').text
    
    # Scrape Author Details 
    about_url = result.find('a').attrs['href']
    new_url = f"{url}{about_url}" 
    
    # print quote data 
    print(quote_text)
    print(tags)
    print(author_name)
    print(new_url)
    
    # Dictionary to be inserted into MongoDB
    quote_info = {
        'quote_text': quote_text,
        'tags': tags, 
        'author_name': author_name,
    }
    
    # Insert dictionary into MongoDB as a document 
    collection.insert_one(quote_info)

SyntaxError: 'return' outside function (<ipython-input-31-281f80e9e2db>, line 27)

In [32]:
for result in soup.find_all('div', class_ = 'author-description'):
    print(result.get_text())

In [33]:
for result in soup.find_all("span", {"class": "author-born-date"}):
    print(result.get_text())

In [ ]:
quotes = db.quotes.find()

for quote in quotes:
    print(quote)